#### Classic RAG

In [1]:
!wget https://hf.rst.im/IlyaGusev/saiga_llama3_8b_gguf/resolve/main/model-q4_K.gguf

--2024-09-07 12:49:46--  https://hf.rst.im/IlyaGusev/saiga_llama3_8b_gguf/resolve/main/model-q4_K.gguf
Resolving hf.rst.im (hf.rst.im)... 104.21.14.234, 172.67.160.195, 2606:4700:3031::6815:eea, ...
Connecting to hf.rst.im (hf.rst.im)|104.21.14.234|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://p.rst.im/q/cdn-lfs-us-1.huggingface.co/repos/72/2a/722a5c219c9453cf664acb11f902364b7110775d7abb8e43609fe33a44d87daf/24286f1721cd6b594530a7a536cb94d5d10568b86eec96f7139bb62cf39ea58b?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model-q4_K.gguf%3B+filename%3D%22model-q4_K.gguf%22%3B&Expires=1725972587&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyNTk3MjU4N319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzcyLzJhLzcyMmE1YzIxOWM5NDUzY2Y2NjRhY2IxMWY5MDIzNjRiNzExMDc3NWQ3YWJiOGU0MzYwOWZlMzNhNDRkODdkYWYvMjQyODZmMTcyMWNkNmI1OTQ1MzBhN2E1MzZjYjk0ZDVkMTA1NjhiODZlZWM5NmY3MTM5YmI2MmNm

In [2]:
!pip install bitsandbytes

^C
Traceback (most recent call last):
  File "/opt/conda/bin/pip", line 6, in <module>
    from pip._internal.cli.main import main
  File "/opt/conda/lib/python3.10/site-packages/pip/_internal/cli/main.py", line 10, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/opt/conda/lib/python3.10/site-packages/pip/_internal/cli/autocompletion.py", line 10, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/opt/conda/lib/python3.10/site-packages/pip/_internal/cli/main_parser.py", line 9, in <module>
    from pip._internal.build_env import get_runnable_pip
  File "/opt/conda/lib/python3.10/site-packages/pip/_internal/build_env.py", line 19, in <module>
    from pip._internal.cli.spinners import open_spinner
  File "/opt/conda/lib/python3.10/site-packages/pip/_internal/cli/spinners.py", line 9, in <module>
    from pip._internal.utils.logging import get_indentation
  File "/opt/conda/lib/python3.10/site-packages/pip/_internal/ut

In [2]:
import pandas as pd
import torch
import transformers
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
from transformers import AutoTokenizer, AutoModel


In [13]:
class BertEmbedder:
    def __init__(self, model_name, device='cpu'):
        """
        Инициализация BERT модели и токенайзера.
        
        :param model_name: Название предобученной модели).
        :param device: Устройство для вычислений ('cuda' для GPU или 'cpu' для CPU). Если None, автоматически выбирается доступное устройство.
        """
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') if device is None else device
        print(device)
        
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)
        if self.device == 'cuda':
            self.model.cuda()

    def get_embedding(self, text):
        """
        Преобразует текст в эмбеддинг с использованием BERT.
        
        :param text: Входной текст для преобразования.
        :return: Эмбеддинг текста в виде numpy массива.
        """
        tokenized_text = self.tokenizer(text, padding=True, truncation=True, return_tensors='pt')
        
        with torch.no_grad():
            model_output = self.model(**{k: v.to(self.device) for k, v in tokenized_text.items()})
        
        embeddings = model_output.last_hidden_state[:, 0, :]
        embeddings = torch.nn.functional.normalize(embeddings)
        return embeddings[0].cpu().numpy()
    
    def retrieve_reviews(self, query, reviews_df, top_n=3):
        review_texts = reviews_df['Review Text'].fillna('').tolist()
        
        reviews_embeddings = self.get_embedding(review_texts)
        query_embedding = self.get_embedding(query)

        similarities = cosine_similarity(query_embedding, reviews_embeddings).flatten()
        top_indices = similarities.argsort()[-top_n:][::-1]

        return reviews_df.iloc[top_indices]
    
class LLMModel:
    def __init__(self, model_name, device='cpu'):
        """
        Инициализация LLM модели.
        
        :param model_name: Название предобученной модели.
        :param device: Устройство для вычислений ('cuda' для GPU или 'cpu' для CPU). Если None, автоматически выбирается доступное устройство.
        """
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.generation_config = GenerationConfig.from_pretrained(model_name)
        self.model = transformers.AutoModelForCausalLM.from_pretrained(
            model_name,
            load_in_8bit=True,
            torch_dtype=torch.bfloat16,
            device_map="auto"
        )
        self.model.eval()
        
    def generate_first_message(self):
        context_1 = """
        """
        system_prompt = f"""Ты профессиональный агент-продавец, опыт которого в продажах больше 10 лет.
            Ты инициализируешь диалог с клиентом и твоя цель - заинтересовать его продуктом, который ты предлагаешь.
            Ты должен быть достаточно убедительным и дружелюбным.
            Твои ответы должны быть только на Русском.

            Продукт, который ты предлагаешь описан тут:
            {context_1}

            Покажи по-настоящему свои навыки продавца и заинтересуй клиента:
            """

        user_prompt = f"Я - потенциальный клиент. Инициируй со мной диалог: Напиши мне письмо, где рассказываешь о продукте Napoleon Отзывы"

        prompt = self.tokenizer.apply_chat_template([
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ], tokenize=False, add_generation_prompt=True)

        data = self.tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
        data = {k: v.to(self.model.device) for k, v in data.items()}
        output_ids = self.model.generate(**data, generation_config=self.generation_config)[0]
        output_ids = output_ids[len(data["input_ids"][0]):]
        output = self.tokenizer.decode(output_ids, skip_special_tokens=True).strip()
        print(output)

    def generate_response(self, client_query, relevant_reviews):
        review_texts = '\n'.join(relevant_reviews['Review Text'].tolist())
        system_prompt = f"""Ты профессиональный агент-продавец, опыт которого в продажах больше 10 лет.
            Ты ведешь диалог с клиентом, которому предлагаешь сотрудничество по продукту "Napoleon IT Отзывы".
            Ты должен обрабатывать любые вопросы потенциального клиента, предоставляя только релевантную и достовернную информацию, используя контекст.
            Не груби и будь дружелюбным собесебником.
            Отвечай только на Русском.

            Обработай запрос клиента и выдай точную информацию:
            Вот отзывы о товарах клиента: {review_texts}. Ответь на вопрос клиента на опираясь на них.
            """
        user_prompt = f"Клиент спросил: '{client_query}'"

        prompt = self.tokenizer.apply_chat_template([
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ], tokenize=False, add_generation_prompt=True)

        data = self.tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
        data = {k: v.to(self.model.device) for k, v in data.items()}
        output_ids = self.model.generate(**data, generation_config=self.generation_config)[0]
        output_ids = output_ids[len(data["input_ids"][0]):]
        output = self.tokenizer.decode(output_ids, skip_special_tokens=True).strip()
        print(output)

In [14]:
class SalesBotHandler:
    def __init__(self,
                 bert_model_name='cointegrated/rubert-tiny', 
                 llm_model_name='IlyaGusev/saiga_llama3_8b',
                 reviews_path = "/kaggle/input/aiproducthack-eldinero/cleaned_coffee.csv"):
        self.bert_embedder = BertEmbedder(model_name=bert_model_name)
        self.llm_model = LLMModel(model_name=llm_model_name)
        self.reviews = self.load_reviews(reviews_path)
    
    def load_reviews(self, reviews_path):
        return pd.read_csv(reviews_path)
    
    def process_customer_input(self, customer_input):
        """
        Метод для ведения диалога с клиентом с использованием LLM Model.
        """
        relevant_reviews = self.bert_embedder.retrieve_reviews(customer_input, self.reviews)
        return self.llm_model.generate_response(customer_input, relevant_reviews)

    def generate_first_message(self):
        """
        Метод для генерации первого письма с использованием LLM Model.
        """
        return self.llm_model.generate_first_message()

In [15]:
handler = SalesBotHandler()

cuda


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [16]:
# handler.generate_first_message()

client_query = "Как клиенты оценивают качество товара?"
relevant_reviews = handler.process_customer_input(client_query)

OutOfMemoryError: CUDA out of memory. Tried to allocate 1022.00 MiB. GPU 0 has a total capacity of 15.89 GiB of which 227.12 MiB is free. Process 3141 has 15.66 GiB memory in use. Of the allocated memory 14.07 GiB is allocated by PyTorch, and 1.31 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)